In [28]:
import pandas as pd
import copy
from sklearn.compose import ColumnTransformer

In [29]:
pip install mglearn

Note: you may need to restart the kernel to use updated packages.


In [30]:
import os
import pandas as pd
import mglearn
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore')

In [31]:
data = pd.read_excel("아이템기반최종데이터.xlsx")
data = data[['상품명', '카테고리', '패턴재질', '모양', '계절', '스타일']]
data=data.drop_duplicates(['상품명'])
data.head()

,상품명,카테고리,패턴재질,모양,계절,스타일
0,[DAYONE LABEL]렌프 자켓원피스(벨트set),미니원피스,무지,단추,겨울,모던시크
1,[DAYONE LABEL]로티 숏팬츠,반바지,무지,포켓,봄 여름,심플베이직
2,[DAYONE LABEL]슬로 보트넥티,티셔츠,무지,보트넥,봄 가을,청순
3,[DAYONE LABEL]워프 스퀘어롱원피스,롱원피스,데님,크롭,여름,청순
4,[DAYONE LABEL]케이즈 하이스키니진 (히든밴딩),스키니,무지,슬림,겨울,캐주얼


In [32]:
data.index

Int64Index([   0,    1,    2,    3,    4,    6,    8,    9,   10,   12,
            ...
            3488, 3490, 3492, 3493, 3495, 3498, 3499, 3500, 3501, 3502],
           dtype='int64', length=1845)

In [33]:
data.index=pd.Series(range(1845),index=data.index)
print(data.index)

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844],
           dtype='int64', length=1845)


# 벡터화 

In [34]:
df = pd.get_dummies(data = data, columns = ['스타일'], prefix = '스타일')
df2 = pd.get_dummies(data = df, columns = ['계절'], prefix = '계절')
df3 = pd.get_dummies(data = df2, columns = ['카테고리'], prefix = '카테고리')
df4 = pd.get_dummies(data = df3, columns = ['모양'], prefix = '모양')
df5 = pd.get_dummies(data = df4, columns = ['패턴재질'], prefix = '패턴재질')
df5.columns

Index(['상품명', '스타일_데일리', '스타일_러블리', '스타일_로맨틱', '스타일_모던', '스타일_모던시크', '스타일_미니멀',
       '스타일_볼드', '스타일_섹시', '스타일_스포티',
       ...
       '패턴재질_퀼팅', '패턴재질_크랙', '패턴재질_털', '패턴재질_트위드', '패턴재질_퍼', '패턴재질_펀칭',
       '패턴재질_프린팅', '패턴재질_플라워', '패턴재질_하찌', '패턴재질_헤링본'],
      dtype='object', length=207)

## 상품명컬럼 인덱스로 바꾸기 

In [35]:
df5.set_index('상품명', inplace = True)

## 코사인유사도 구하기

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
genre_c_sim = cosine_similarity(df5,df5).argsort()[:,::-1] 


In [38]:
def get_recommend_item_list(df,상품명,top=1800): 
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품명']==상품명].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index=genre_c_sim[target_item_index,:top].reshape(-1)
  
    #본인 제외
    sim_index=sim_index[sim_index != target_item_index]

    #data frame로 만들고 vote_count로 정렬한뒤 return
    result=df.iloc[sim_index]
    results=result[['상품명','스타일','계절','카테고리','모양','패턴재질']]
    results =results.drop_duplicates(['상품명'])
    
    return results[:20]

In [41]:
get_recommend_item_list(data,'[La belle rose]언더 베이직 유넥티')

,상품명,스타일,계절,카테고리,모양,패턴재질
748,후니 골지 유넥티,심플베이직,봄 가을,티셔츠,유넥,무지
750,휴즈 여리핏 롱슬리브티,심플베이직,봄 가을,티셔츠,유넥,무지
391,[DAYONE LABEL]에브 골지유넥티,심플베이직,봄 가을,티셔츠,유넥,골지
1805,플럽 단추 유넥티,심플베이직,봄 여름 가을,티셔츠,유넥,무지
624,오데썽 와이드 스퀘어긴팔티,심플베이직,봄 가을,티셔츠,스퀘어넥,무지
474,라임 유넥티,심플베이직,여름,티셔츠,유넥,무지
641,위즈더 크롭 트임티,심플베이직,봄 가을,티셔츠,트임,무지
642,위켄더 롱슬리브티,심플베이직,봄 가을,티셔츠,라운드넥,무지
1685,워너블 베이직셔츠,심플베이직,봄 가을,티셔츠,카라,무지
651,유즈 양면 유넥티,심플베이직,여름,티셔츠,유넥,무지


In [13]:
target_item_index=df[df['상품명']=='[La belle rose]언더 베이직 유넥티'].index.values

In [15]:
sim_index=genre_c_sim[target_item_index,:].reshape(-1)

In [16]:
sim_index=sim_index[sim_index != target_item_index]

<ipython-input-16-034e193bde15>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  sim_index=sim_index[sim_index != target_item_index]


In [26]:
sim_index=sim_index

In [27]:
result=df.iloc[sim_index]

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [10]:
def get_recommend_item_list2(df,상품명,top=1800): 
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품명']==상품명].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_index1=genre_c_sim1[target_item_index,:top].reshape(-1)
  
    #본인 제외
    sim_index1=sim_index1[sim_index1 != target_item_index]

    #data frame로 만들고 vote_count로 정렬한뒤 return
    result1=df.iloc[sim_index1]
    results1=result1[['상품명','스타일','계절','카테고리','모양','패턴재질']]
    results1=results1.drop_duplicates()

    return results1